In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
import json

In [3]:
sc = SparkContext('local[8]', 'task2')

In [4]:
reviews_txt = sc.textFile('asnlib/publicdata/review.json')


In [5]:
business_txt = sc.textFile('asnlib/publicdata/business.json')

In [6]:
reviews_json = reviews_txt.map(lambda x: json.loads(x))

In [ ]:
reviews_json.first()

In [ ]:
business_json = business_txt.map(lambda x: json.loads(x))

In [ ]:
business_json.take(1)

In [ ]:
def get_category_business_id(row):
    category_business_list = []
    for category in row['categories'].split(','):
        category_business_list.append((category.strip(), row['business_id']))
    return category_business_list

In [ ]:
# categories = business_json.map(lambda x: ([cat.strip()for cat in x['categories'].split(',')], x['business_id']))


In [ ]:
categories = business_json.filter(lambda x: x.get('categories',  None) is not None).flatMap(get_category_business_id).groupByKey().map(lambda x: (x[0], list(x[1])))

In [ ]:
categories.first()

In [ ]:
reduced_cat = categories.reduceByKey(lambda a,b: a + b)

In [ ]:
reduced_cat.collect()

In [ ]:
business_ratings = reviews_json.map(lambda x: (x['business_id'], x['stars']))

In [ ]:
business_ratings.count()

In [ ]:
grouped_b = business_ratings.groupByKey().map(lambda x: (x[0], list(x[1])))

In [ ]:
grouped_b.take(10)

In [ ]:
joined = business_ratings.join(categories)

In [ ]:
joined.first()

In [ ]:
categories = business_json.filter(lambda x: x.get('categories',  None) is not None).flatMap(get_category_business_id)

In [ ]:
categories.take(10)

In [ ]:
joined_rdd = categories.join(review_stars)

In [ ]:
joined_rdd.take(10)

In [ ]:
category_ratings_map = joined_rdd.map(lambda x: x[1]).groupByKey().map(lambda x: (x[0], list(x[1])))

In [ ]:
average_ratings = category_ratings_map.mapValues(lambda x: sum(x)/len(x)).sortBy(lambda x: (-x[1], x[0]))

In [ ]:
average_ratings.take(50)